In [7]:
import os
import re
from mutagen.easyid3 import EasyID3
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

# Configura tus credenciales de Spotify
SPOTIPY_CLIENT_ID = 'your_spotify_client_id'
SPOTIPY_CLIENT_SECRET = 'your_spotify_client_secret'

def get_spotify_client():
    auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
    return Spotify(auth_manager=auth_manager)

def extract_artist_and_track(file_name):
    match = re.match(r'(.+?) - (.+)\.mp3', file_name)
    if match:
        return match.group(1), match.group(2)
    return None, None

def search_track_on_spotify(sp, artist_name, track_name):
    query = f'artist:{artist_name} track:{track_name}'
    results = sp.search(q=query, type='track', limit=1)
    tracks = results.get('tracks', {}).get('items', [])
    if tracks:
        return tracks[0]
    return None

def update_album_metadata(file_path, album_name):
    audio = EasyID3(file_path)
    audio['album'] = album_name
    audio.save()

def process_folder(folder_path):
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The folder {folder_path} does not exist.")
    
    sp = get_spotify_client()
    
    for file in os.listdir(folder_path):
        if file.endswith(".mp3"):
            file_path = os.path.join(folder_path, file)
            artist_name, track_name = extract_artist_and_track(file)
            
            if artist_name and track_name:
                track_metadata = search_track_on_spotify(sp, artist_name, track_name)
                
                if track_metadata:
                    album_name = track_metadata['album']['name']
                    update_album_metadata(file_path, album_name)
                    print(f"Updated album metadata for '{file}'.")
                else:
                    print(f"Could not find metadata for '{file}'.")
            else:
                print(f"Skipping file '{file}' due to incorrect format.")

def main():
    folder_path = input("Enter the path to the folder with MP3 files: ")
    process_folder(folder_path)

if __name__ == '__main__':
    main()

Could not find metadata for 'Tam Tam Go! - Piel Sobre Piel (feat. Maico).mp3'.
Could not find metadata for 'Tam Tam Go! - Belleza Pasajera (feat. Coti).mp3'.
Updated album metadata for 'Tam Tam Go! - Atrapados En La Red.mp3'.
Could not find metadata for 'Tam Tam Go! - Caer a Tus Pies (feat. Mikel Erentxun).mp3'.
Updated album metadata for 'Tam Tam Go! - Lucía De Los Cartones.mp3'.
Could not find metadata for 'Tam Tam Go! - Pasarán (feat. Antonio Vega).mp3'.
Could not find metadata for 'Tam Tam Go! - Espaldas Mojadas (feat. Mikel Izal).mp3'.
Could not find metadata for 'Tam Tam Go! - Manuel Raquel (feat. Manuela Vellés).mp3'.
Updated album metadata for 'Tam Tam Go! - I Come For You.mp3'.
Could not find metadata for 'Tam Tam Go! - Un Juramento Entre Tú y Yo (feat. Txetxu Altube).mp3'.
